In [3]:
import os
import glob
from datetime import datetime
from netCDF4 import Dataset
import netCDF4 as nc
from datetime import datetime
import pandas as pd
import numpy as np
import xarray as xr
import sys
sys.path.append('C:/Users/Administrator/anaconda3/envs/gdal/Lib/site-packages/')
sys.path.append('C:/Users/Administrator/anaconda3/envs/gdal/Lib/site-packages/lmoments3')
import distr
from scipy import stats
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm import tqdm
import basefunc

In [4]:
#ReturnPeriod_change###


def f_normal(inputs):
    """
    
                MV et.al return period/year, LV design return period/year
    high income：100, 30
    upper middle income：50, 10
    low middle/low income：10, 2
    
    inputs[0]: income group index. 1: low income group, 2: lower middle income group, 3: upper middle income group, 4: high income group
    inputs[1]: assets_type. 1: MV,  2: LV
    inputs[2:]: new return period in the future. 0: 1.1years, 1: 2years, 2: 5years, 3: 10years, 4: 20years, 5: 30years, 6: 50years, 7: 100years, 8:200year 
    
    output: the proportion of new return period to the historical design return period
    """
    income=inputs[0]
    assets_type=inputs[1]
    new_rt=inputs[-9:]
    y=-1.0
    if assets_type==1:
        if income==1 or income==2: 
            y=(new_rt[3]-10.0)/10.0                           
        elif income==3: 
            y=(new_rt[6]-50.0)/50.0 
        elif  income==4: 
            y=(new_rt[7]-100.0)/100.0  
        else:
            y=-1.0   
    elif assets_type==2:
        if income==1 or income==2:
            y=(new_rt[1]-2.0)/2.0 
        elif income==3:
            y=(new_rt[3]-10.0)/10.0             
        elif income==4:    
            y=(new_rt[5]-30.0)/30.0
        else:
            y=-1.0
  
    return  y

def f_strong(inputs):
    """
    
    
                return period/years: MV, LV
    [1.1,2,5,10,20,30,50,100,200]
    high income：200,50
    upper middle income：100,30
    low middle/low income：30,10
    
    inputs[0]: income group index. 1: low income group, 2: lower middle income group, 3: upper middle income group, 4: high income group
    inputs[1]: assets_type. 1: MV,  2: LV
    inputs[2:]: new return period in the future. 0: 1.1years, 1: 2years, 2: 5years, 3: 10years, 4: 20years, 5: 30years, 6: 50years, 7: 100years, 8:200year 
    
    output: the proportion of new return period to the historical design return period
    """
    income=inputs[0]
    assets_type=inputs[1]
    new_rt=inputs[-9:]
    y=-1.0
    if assets_type==1:
        if income==1 or income==2: 
            y=(new_rt[5]-30)/30                            
        elif income==3: 
            y=(new_rt[7]-100)/100  
        elif  income==4: 
            y=(new_rt[8]-200)/200 
        else:
            y=-1.0    
    elif assets_type==2:
        if income==1 or income==2:
            y=(new_rt[3]-10)/10 
        elif income==3:
            y=(new_rt[5]-30)/30        
        elif income==4:    
            y=(new_rt[6]-50)/50
        else:
            y=-1.0
    return  y

# main
types=['MV','LV']
SCENARIO_YEARS = {
    
    'ssp126': [(2030, 2060), (2070, 2100)],
    'ssp245': [(2030, 2060), (2070, 2100)],
    'ssp370': [(2030, 2060), (2070, 2100)],
    'ssp585': [(2030, 2060), (2070, 2100)],
}

for name in types:
    for scen, ranges in SCENARIO_YEARS.items():
        for time_low,time_up in ranges:
            if name=='MV':
                assets_type=1
            if name=='LV':
                assets_type=2
           
            assets_type=np.full((1,600,1440),assets_type)
            income=basefunc.getRaster("./data/income_group.tif")[:600,:]
            
            income = income.reshape(1,600,1440)

            new_rt = basefunc.getRaster('./results_for_gev/newRT_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')
            
            array=np.concatenate((income,assets_type,new_rt),axis=0)
            rt_change=np.apply_along_axis(f_normal,0,array)
            rt_change=rt_change.reshape(1,600,1440)

            length=basefunc.getRaster(f'./{name}.tif')
            length=length.reshape(1,600,1440)
            out=np.concatenate((length,rt_change),axis=0)

            ref_tif='./data/ref.tif'
            output_path='./results_for_rt_change/f_normal/'+name+'_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif'
    
            basefunc.array2Raster(out,ref_tif,output_path)


